In [1]:
%matplotlib inline
import csv
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np 
import keras
from keras.models import Sequential 
from keras.layers.pooling import MaxPooling2D
from keras.layers import Flatten, Dense ,Lambda, Dropout
from keras.layers.convolutional import Convolution2D, Cropping2D
from keras.optimizers import Adam

Using TensorFlow backend.


We try to make the data set bigger by using one of the following methods?
1. collect more traning data we can drive the road in the opst direction
2. flip the images and flip the stereeing images

How we can stay in the center of the road?
1. collect more training data
2. use the left and rigth images. From left image we see that the car should drive to right ...

How we can make the modell better?
1. crope the part of pixel wich we don't need
2. Use another Network architecture

In [2]:
lines = []
with open ('./data-udacity/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

In [3]:
# the images and the measurments from the data set
sim_images = []
sim_measurments = []
for line in lines[0:1000]:
    for i in range(3):
        source_path = line[i]
        tokens = source_path.split('/')
        filename = tokens[-1]
        local_path = "./data-udacity/IMG/"+filename
        image = cv2.imread(local_path)
        sim_images.append(image)
    # correcction factor    
    correcction = 0.2
    measurment = float(line[3])
    # measurmaent for the center image
    sim_measurments.append(measurment)
    # measurmaent for the left image + correcction factor
    sim_measurments.append(measurment + correcction) 
    # measurmaent for the right image-  correcction factor
    sim_measurments.append(measurment - correcction)
# the image and measurments data set: 
# image = fliped_image +  simu_images
images =  []
measurments =  []
for image, measurment in zip(sim_images, sim_measurments):
    # flip the image about the verticle axes 
    fliped_image = cv2.flip(image,1)
    fliped_measurment =measurment*-1.0
    images.append(image)
    images.append(fliped_image)
    measurments.append(measurment)
    measurments.append(fliped_measurment)
    
    

keras accepts numpy file

In [4]:
X_train = np.array(images)
y_train = np.array(measurments)
print(len(X_train))
print(len(y_train))

6000
6000


## Regeression Task

In [5]:
model = Sequential()
# colored images
model.add(Lambda(lambda x: x/ 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Flatten())
# one single output layer 
model.add(Dense(1))
# cost  function
model.compile(optimizer='adam', loss = 'mse')
model.fit(X_train,y_train, validation_split=0.2,shuffle=True,nb_epoch=3  )
model.save('modelR.h5')

C:\Users\MKaram\Anaconda\envs\carnd-term1\lib\site-packages\keras\models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 4800 samples, validate on 1200 samples
Epoch 1/3
4800/4800 [==============================] - 8s - loss: 0.4528 - val_loss: 0.0619

## CNN:
LeNet is a convolution neural network consists of 6 convolution filters then max pull which followed by 16 convolution filters and then  max pull layer at least there is fully connected network

In [68]:
model = Sequential()
# colored images
model.add(Lambda(lambda x: x/ 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Convolution2D(6,5,5,activation = 'relu'))
model.add(MaxPooling2D())
model.add(Convolution2D(16,5,5,activation = 'relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(128))
model.add(Dense(84)) 
# one single output layer 
model.add(Dense(1))
# cost  function
model.compile(optimizer='adam', loss = 'mse')
model.fit(X_train,y_train, validation_split=0.2,shuffle=True, nb_epoch=3 )
model.save('modelCNN.h5')

C:\Users\MKaram\Anaconda\envs\carnd-term1\lib\site-packages\ipykernel\__main__.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), activation="relu")`
C:\Users\MKaram\Anaconda\envs\carnd-term1\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (5, 5), activation="relu")`


## End to End Learning:

In [7]:
model = Sequential()
# Normalization layer
model.add(Lambda(lambda x: x/ 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
# 5 x 5 Convolution filters + subsample which make the size of the images smaller
model.add(Convolution2D(24,5,5, subsample = (2,2), activation = 'relu', name='conv1'))
model.add(Convolution2D(36,5,5, subsample = (2,2), activation = 'relu', name='conv2'))
model.add(Convolution2D(48,5,5, subsample = (2,2), activation = 'relu', name='conv3'))
# 3 x 3 Convolution filters
model.add(Convolution2D(64,3,3,activation = 'relu', name='conv4'))
model.add(Convolution2D(64,3,3,activation = 'relu', name='conv5'))
model.add(Flatten())
model.add(Dropout(.8))
model.add(Dense(100))
model.add(Dropout(.8))
model.add(Dense(50)) 
model.add(Dense(10))
# one single output layer 
model.add(Dense(1))
# cost  function
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)
model.compile(optimizer=adam, loss = 'mse')
model.fit(X_train,y_train, validation_split=0.2,shuffle=True, nb_epoch=3 )
model.save('modelEnd.h5')

C:\Users\MKaram\Anaconda\envs\carnd-term1\lib\site-packages\ipykernel\__main__.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), name="conv1", activation="relu", strides=(2, 2))`
C:\Users\MKaram\Anaconda\envs\carnd-term1\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), name="conv2", activation="relu", strides=(2, 2))`
C:\Users\MKaram\Anaconda\envs\carnd-term1\lib\site-packages\ipykernel\__main__.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), name="conv3", activation="relu", strides=(2, 2))`
C:\Users\MKaram\Anaconda\envs\carnd-term1\lib\site-packages\ipykernel\__main__.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), name="conv4", activation="relu")`
C:\Users\MKaram\Anaconda\envs\carnd-term1\lib\site-packages\ipykernel\__main__.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3

Train on 4800 samples, validate on 1200 samples
Epoch 1/3
4800/4800 [==============================] - 89s - loss: 0.0283 - val_loss: 0.0218